In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [2]:
dataset_dir = '../datasets/prosper-loans'

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

In [3]:
%%bash
cd ../datasets/prosper-loans
wget https://s3.amazonaws.com/udacity-hosted-downloads/ud651/prosperLoanData.csv

--2019-08-12 03:43:39--  https://s3.amazonaws.com/udacity-hosted-downloads/ud651/prosperLoanData.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.100.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.100.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86471101 (82M) [application/octet-stream]
Saving to: ‘prosperLoanData.csv’

     0K .......... .......... .......... .......... ..........  0%  138K 10m13s
    50K .......... .......... .......... .......... ..........  0% 6.03M 5m13s
   100K .......... .......... .......... .......... ..........  0%  284K 5m8s
   150K .......... .......... .......... .......... ..........  0% 24.0M 3m51s
   200K .......... .......... .......... .......... ..........  0%  281K 4m5s
   250K .......... .......... .......... .......... ..........  0% 12.5M 3m25s
   300K .......... .......... .......... .......... ..........  0% 11.0M 2m57s
   350K .......... .......... .......... .......... ..........  0%  

In [4]:
df = pd.read_csv(os.path.join(dataset_dir, 'prosperLoanData.csv'))

In [5]:
features = [
    'BorrowerAPR', 'BorrowerRate',
    'EstimatedEffectiveYield', 'EstimatedReturn',
    'ProsperScore', 'CreditScoreRangeUpper',
    'OpenCreditLines', 'AmountDelinquent',
    'StatedMonthlyIncome', 'MonthlyLoanPayment',
    'ClosedDate', 'LoanStatus',
]
df = df[features]

In [6]:
df = df.dropna(subset=['ProsperScore'])
df = df[df['LoanStatus'].isin(['Completed', 'Defaulted', 'Chargedoff'])]
df['LoanStatus'] = df['LoanStatus'] == 'Completed'

In [7]:
g = df.groupby('LoanStatus')
df = g.apply(lambda x: x.sample(g.size().min(), random_state=1)).reset_index(drop=True)

In [8]:
df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])
df = df.sort_values(by='ClosedDate')
del df['ClosedDate']

In [9]:
df = df[[c for c in df if c not in ['LoanStatus']] + ['LoanStatus']]

In [10]:
df.to_csv(os.path.join(dataset_dir, 'prosper-loans.csv'), index=False)